In [1]:
import pandas as pd
import numpy as np
import os
import gc

PATH = '/home/kai/data/kaggle/homecredit/'
print(os.listdir(PATH))

['application_train.csv', 'POS_CASH_balance.csv', 'application_test.csv', 'previous_application.csv', 'credit_card_balance.csv', 'sample_submission.csv', 'installments_payments.csv', 'HomeCredit_columns_description.csv', 'bureau_balance.csv', 'bureau.csv']


# installments payments

In [87]:
installments_payments = pd.read_csv(PATH + 'installments_payments.csv')
print(installments_payments.columns.values, installments_payments.shape)

['SK_ID_PREV' 'SK_ID_CURR' 'NUM_INSTALMENT_VERSION' 'NUM_INSTALMENT_NUMBER'
 'DAYS_INSTALMENT' 'DAYS_ENTRY_PAYMENT' 'AMT_INSTALMENT' 'AMT_PAYMENT'] (13605401, 8)


In [4]:
late_day = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']
print(sum(late_day > 0))

1146669


In [101]:
print(installments_payments.groupby(by='NUM_INSTALMENT_NUMBER')[['SK_ID_PREV']].count())

                       SK_ID_PREV
NUM_INSTALMENT_NUMBER            
1                         1004160
2                          985716
3                          968279
4                          943502
5                          880007
6                          827973
7                          679739
8                          644708
9                          592473
10                         549140
11                         427866
12                         385541
13                         233976
14                         221991
15                         206909
16                         195209
17                         181034
18                         168822
19                         140585
20                         133304
21                         124824
22                         117886
23                         110418
24                          99692
25                          70977
26                          67765
27                          65125
28            

In [99]:
group1 = installments_payments.groupby(by=['SK_ID_PREV'])[['SK_ID_CURR']].count().reset_index()[['SK_ID_CURR']].rename(columns={'SK_ID_CURR':'a'})
group2 = installments_payments.groupby(by=['SK_ID_PREV'])[['NUM_INSTALMENT_VERSION']].nunique().reset_index()[['NUM_INSTALMENT_VERSION']].rename(columns={'NUM_INSTALMENT_VERSION':'a'})
print((group1 - group2).sort_values(by='a',ascending=False))

          a
744296  292
866111  278
12446   247
260231  242
421641  235
393868  222
926704  221
220326  219
85859   218
840801  215
746308  214
342039  213
778063  212
759141  212
392962  210
210867  210
669309  208
834568  208
139478  208
227854  206
735315  206
512604  205
350622  205
577497  204
432241  203
597775  203
579434  202
921287  202
41544   202
935920  202
...     ...
944767    0
106292    0
678399    0
798672    0
798671    0
678396    0
398092    0
600505    0
398102    0
944759    0
106253    0
398111    0
398119    0
615251    0
106246    0
267979    0
398135    0
267997    0
944778    0
568993    0
798611    0
639346    0
398182    0
798606    0
944786    0
106229    0
267980    0
944789    0
639347    0
0         0

[997752 rows x 1 columns]


In [13]:
x = set(installments_payments[installments_payments['DAYS_ENTRY_PAYMENT'].isnull()]['SK_ID_PREV'].values)
y = set(installments_payments[installments_payments['AMT_PAYMENT'].isnull()]['SK_ID_PREV'].values)

print((x-y)&(y-x))

set()


In [15]:
print(set(installments_payments[installments_payments['DAYS_INSTALMENT'].isnull()]['SK_ID_PREV'].values))

set()


In [18]:
c = ['NUM_INSTALMENT_NUMBER', 'DAYS_INSTALMENT', 'DAYS_ENTRY_PAYMENT', 'AMT_INSTALMENT', 'AMT_PAYMENT']
print(installments_payments[installments_payments['SK_ID_PREV']==2360056][c].sort_values('NUM_INSTALMENT_NUMBER'))

          NUM_INSTALMENT_NUMBER  DAYS_INSTALMENT  DAYS_ENTRY_PAYMENT  \
11604504                      1          -2825.0             -2853.0   
13417545                      2          -2853.0             -2853.0   
11585804                      3          -2839.0             -2839.0   
13206058                      4          -2794.0             -2812.0   
12288958                      5          -2812.0             -2812.0   
13223551                      6          -2803.0             -2803.0   
12601880                      7          -2799.0             -2799.0   
12393812                      8          -2763.0             -2777.0   
13351332                      9          -2777.0             -2777.0   
13313085                     10          -2776.0             -2776.0   
12644834                     11          -2771.0             -2771.0   
12923465                     12          -2770.0             -2770.0   
12366119                     13          -2769.0             -27

# previous application

In [2]:
previous_application = pd.read_csv(PATH + 'previous_application.csv')
print(previous_application.columns.values, previous_application.shape)

['SK_ID_PREV' 'SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'AMT_ANNUITY'
 'AMT_APPLICATION' 'AMT_CREDIT' 'AMT_DOWN_PAYMENT' 'AMT_GOODS_PRICE'
 'WEEKDAY_APPR_PROCESS_START' 'HOUR_APPR_PROCESS_START'
 'FLAG_LAST_APPL_PER_CONTRACT' 'NFLAG_LAST_APPL_IN_DAY' 'RATE_DOWN_PAYMENT'
 'RATE_INTEREST_PRIMARY' 'RATE_INTEREST_PRIVILEGED'
 'NAME_CASH_LOAN_PURPOSE' 'NAME_CONTRACT_STATUS' 'DAYS_DECISION'
 'NAME_PAYMENT_TYPE' 'CODE_REJECT_REASON' 'NAME_TYPE_SUITE'
 'NAME_CLIENT_TYPE' 'NAME_GOODS_CATEGORY' 'NAME_PORTFOLIO'
 'NAME_PRODUCT_TYPE' 'CHANNEL_TYPE' 'SELLERPLACE_AREA'
 'NAME_SELLER_INDUSTRY' 'CNT_PAYMENT' 'NAME_YIELD_GROUP'
 'PRODUCT_COMBINATION' 'DAYS_FIRST_DRAWING' 'DAYS_FIRST_DUE'
 'DAYS_LAST_DUE_1ST_VERSION' 'DAYS_LAST_DUE' 'DAYS_TERMINATION'
 'NFLAG_INSURED_ON_APPROVAL'] (1670214, 37)


In [3]:
print(previous_application.isnull().any())

SK_ID_PREV                     False
SK_ID_CURR                     False
NAME_CONTRACT_TYPE             False
AMT_ANNUITY                     True
AMT_APPLICATION                False
AMT_CREDIT                      True
AMT_DOWN_PAYMENT                True
AMT_GOODS_PRICE                 True
WEEKDAY_APPR_PROCESS_START     False
HOUR_APPR_PROCESS_START        False
FLAG_LAST_APPL_PER_CONTRACT    False
NFLAG_LAST_APPL_IN_DAY         False
RATE_DOWN_PAYMENT               True
RATE_INTEREST_PRIMARY           True
RATE_INTEREST_PRIVILEGED        True
NAME_CASH_LOAN_PURPOSE         False
NAME_CONTRACT_STATUS           False
DAYS_DECISION                  False
NAME_PAYMENT_TYPE              False
CODE_REJECT_REASON             False
NAME_TYPE_SUITE                 True
NAME_CLIENT_TYPE               False
NAME_GOODS_CATEGORY            False
NAME_PORTFOLIO                 False
NAME_PRODUCT_TYPE              False
CHANNEL_TYPE                   False
SELLERPLACE_AREA               False
N

In [4]:
print(set(previous_application['NAME_CONTRACT_STATUS'].values))

{'Unused offer', 'Refused', 'Canceled', 'Approved'}


In [8]:
x = previous_application[previous_application['NAME_CONTRACT_STATUS']=='Canceled']
y = x[~x['AMT_ANNUITY'].isnull()]
print(y.isnull().any())

SK_ID_PREV                     False
SK_ID_CURR                     False
NAME_CONTRACT_TYPE             False
AMT_ANNUITY                    False
AMT_APPLICATION                False
AMT_CREDIT                     False
AMT_DOWN_PAYMENT                True
AMT_GOODS_PRICE                 True
WEEKDAY_APPR_PROCESS_START     False
HOUR_APPR_PROCESS_START        False
FLAG_LAST_APPL_PER_CONTRACT    False
NFLAG_LAST_APPL_IN_DAY         False
RATE_DOWN_PAYMENT               True
RATE_INTEREST_PRIMARY           True
RATE_INTEREST_PRIVILEGED        True
NAME_CASH_LOAN_PURPOSE         False
NAME_CONTRACT_STATUS           False
DAYS_DECISION                  False
NAME_PAYMENT_TYPE              False
CODE_REJECT_REASON             False
NAME_TYPE_SUITE                 True
NAME_CLIENT_TYPE               False
NAME_GOODS_CATEGORY            False
NAME_PORTFOLIO                 False
NAME_PRODUCT_TYPE              False
CHANNEL_TYPE                   False
SELLERPLACE_AREA               False
N

In [14]:
print(previous_application[['DAYS_FIRST_DUE','DAYS_LAST_DUE_1ST_VERSION','DAYS_LAST_DUE']])

         DAYS_FIRST_DUE  DAYS_LAST_DUE_1ST_VERSION  DAYS_LAST_DUE
0                 -42.0                      300.0          -42.0
1                -134.0                      916.0       365243.0
2                -271.0                       59.0       365243.0
3                -482.0                     -152.0         -182.0
4                   NaN                        NaN            NaN
5                -654.0                     -144.0         -144.0
6                   NaN                        NaN            NaN
7                   NaN                        NaN            NaN
8                   NaN                        NaN            NaN
9                   NaN                        NaN            NaN
10               -705.0                      885.0         -345.0
11               -785.0                       85.0         -725.0
12               -830.0                     -140.0         -200.0
13               -378.0                     -168.0         -168.0
14        

# POS_CASH balance

In [15]:
POS_CASH_balance = pd.read_csv(PATH + 'POS_CASH_balance.csv')
print(POS_CASH_balance.columns.values, POS_CASH_balance.shape)

['SK_ID_PREV' 'SK_ID_CURR' 'MONTHS_BALANCE' 'CNT_INSTALMENT'
 'CNT_INSTALMENT_FUTURE' 'NAME_CONTRACT_STATUS' 'SK_DPD' 'SK_DPD_DEF'] (10001358, 8)


In [65]:
print(POS_CASH_balance.isnull().any())

SK_ID_PREV               False
SK_ID_CURR               False
MONTHS_BALANCE           False
CNT_INSTALMENT            True
CNT_INSTALMENT_FUTURE     True
NAME_CONTRACT_STATUS     False
SK_DPD                   False
SK_DPD_DEF               False
dtype: bool
         SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  CNT_INSTALMENT  \
767082      1775141      244572             -96             4.0   
3573052     1595167      454443             -96            10.0   
5231758     2588917      294677             -96             4.0   
2251801     1810403      145248             -96            10.0   
727339      1925384      330163             -96            12.0   
758271      2247724      363480             -96            12.0   
727341      2167952      186204             -96            12.0   
2564534     2700909      308347             -96            10.0   
5231753     1475098      360965             -96             3.0   
2251811     2140266      243873             -96             4.0   
256

In [77]:
print(POS_CASH_balance[POS_CASH_balance['SK_ID_PREV']==2593329].sort_values(by = 'MONTHS_BALANCE'))

         SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  CNT_INSTALMENT  \
5559520     2593329      409175             -47            18.0   
7294645     2593329      409175             -46            18.0   
6484940     2593329      409175             -45            18.0   
2981150     2593329      409175             -44            18.0   
9092469     2593329      409175             -43            18.0   
2985779     2593329      409175             -42            18.0   
1227261     2593329      409175             -41            18.0   
4758696     2593329      409175             -40            18.0   
8047934     2593329      409175             -39            18.0   
3763248     2593329      409175             -38             9.0   

         CNT_INSTALMENT_FUTURE NAME_CONTRACT_STATUS  SK_DPD  SK_DPD_DEF  
5559520                   18.0               Active       0           0  
7294645                   17.0               Active       0           0  
6484940                   16.0          

In [31]:
x = POS_CASH_balance[POS_CASH_balance['NAME_CONTRACT_STATUS']=='Signed'][['MONTHS_BALANCE','CNT_INSTALMENT','CNT_INSTALMENT_FUTURE']]

print(x[POS_CASH_balance['CNT_INSTALMENT'] - POS_CASH_balance['CNT_INSTALMENT_FUTURE'] != 0].shape)
# print(x)

(25897, 3)


/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [80]:
x = POS_CASH_balance.groupby(by='CNT_INSTALMENT_FUTURE')[['SK_DPD']].count().reset_index().rename(columns={'SK_DPD':'count'}).sort_values(by='count', ascending=False)
print(x)

    CNT_INSTALMENT_FUTURE    count
0                     0.0  1185960
6                     6.0   614058
4                     4.0   613632
5                     5.0   600295
3                     3.0   582007
2                     2.0   547199
1                     1.0   512279
10                   10.0   481390
8                     8.0   480167
7                     7.0   472665
9                     9.0   467606
12                   12.0   389585
11                   11.0   376430
18                   18.0   165735
17                   17.0   159457
16                   16.0   157022
15                   15.0   151212
14                   14.0   150588
13                   13.0   143572
24                   24.0   136742
23                   23.0   130810
22                   22.0   126621
21                   21.0   122275
20                   20.0   119339
19                   19.0   115024
36                   36.0    55228
30                   30.0    53102
35                  

In [20]:
print(POS_CASH_balance[['CNT_INSTALMENT_FUTURE', 'NAME_CONTRACT_STATUS', 'SK_DPD', 'SK_DPD_DEF']])

          CNT_INSTALMENT_FUTURE NAME_CONTRACT_STATUS  SK_DPD  SK_DPD_DEF
0                          45.0               Active       0           0
1                          35.0               Active       0           0
2                           9.0               Active       0           0
3                          42.0               Active       0           0
4                          35.0               Active       0           0
5                          12.0               Active       0           0
6                          43.0               Active       0           0
7                          36.0               Active       0           0
8                          12.0               Active       0           0
9                          16.0               Active       0           0
10                         12.0               Active       0           0
11                         12.0               Active       0           0
12                         36.0               Activ

# credit card balance

In [36]:
credit_card_balance = pd.read_csv(PATH + 'credit_card_balance.csv')
print(credit_card_balance.shape)
print(credit_card_balance.isnull().any())

(3840312, 23)
SK_ID_PREV                    False
SK_ID_CURR                    False
MONTHS_BALANCE                False
AMT_BALANCE                   False
AMT_CREDIT_LIMIT_ACTUAL       False
AMT_DRAWINGS_ATM_CURRENT       True
AMT_DRAWINGS_CURRENT          False
AMT_DRAWINGS_OTHER_CURRENT     True
AMT_DRAWINGS_POS_CURRENT       True
AMT_INST_MIN_REGULARITY        True
AMT_PAYMENT_CURRENT            True
AMT_PAYMENT_TOTAL_CURRENT     False
AMT_RECEIVABLE_PRINCIPAL      False
AMT_RECIVABLE                 False
AMT_TOTAL_RECEIVABLE          False
CNT_DRAWINGS_ATM_CURRENT       True
CNT_DRAWINGS_CURRENT          False
CNT_DRAWINGS_OTHER_CURRENT     True
CNT_DRAWINGS_POS_CURRENT       True
CNT_INSTALMENT_MATURE_CUM      True
NAME_CONTRACT_STATUS          False
SK_DPD                        False
SK_DPD_DEF                    False
dtype: bool


In [85]:
x = credit_card_balance.groupby(by='NAME_CONTRACT_STATUS')[['SK_DPD']].count().reset_index().rename(columns={'SK_DPD':'count'}).sort_values(by='count', ascending=False)
print(x)

  NAME_CONTRACT_STATUS    count
0               Active  3698436
2            Completed   128918
6               Signed    11058
3               Demand     1365
5        Sent proposal      513
4              Refused       17
1             Approved        5


In [84]:
c = ['AMT_DRAWINGS_ATM_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT', 'AMT_DRAWINGS_POS_CURRENT']

x = (credit_card_balance[c].sum(axis=1) - credit_card_balance['AMT_DRAWINGS_CURRENT']).to_frame()
print(x[x[x.columns.values[0]]!=0])

                    0
53      -2.145780e+03
418     -2.910383e-11
650     -2.700365e+04
847     -1.293332e+04
1595    -2.414970e+03
1608     7.275958e-12
1827    -1.455192e-11
1964    -1.350000e+00
2106    -1.218060e+03
2589    -1.336950e+02
2622    -1.049580e+03
2778     9.094947e-13
2779     9.094947e-13
2938     3.637979e-12
3006    -1.719000e+01
3035    -1.346400e+03
3084    -4.837500e+02
3128    -2.355750e+02
3418    -2.609100e+02
3435    -2.772450e+02
3436    -1.041300e+02
3742    -7.275958e-12
3788    -8.097750e+03
3905     1.455192e-11
4139    -5.400000e+02
4909    -1.333350e+02
4919    -2.335500e+01
5938    -1.104350e+04
6511    -1.559430e+03
6757    -7.275958e-12
...               ...
3821378 -6.853500e+01
3821424 -6.750000e+01
3821431 -1.319850e+02
3821448  3.637979e-12
3821486 -1.325700e+02
3821496 -2.910383e-11
3821536 -1.229850e+02
3821555 -1.398600e+02
3821588 -5.580000e+00
3822418 -9.607500e+01
3822499  1.818989e-12
3822978 -7.560000e+02
3826034 -1.708560e+03
3828368 -1

In [52]:
print(credit_card_balance[credit_card_balance['AMT_PAYMENT_CURRENT'] < credit_card_balance['AMT_PAYMENT_TOTAL_CURRENT']][['AMT_PAYMENT_CURRENT','AMT_PAYMENT_TOTAL_CURRENT']])

Empty DataFrame
Columns: [AMT_PAYMENT_CURRENT, AMT_PAYMENT_TOTAL_CURRENT]
Index: []


In [60]:
print(credit_card_balance[credit_card_balance['SK_DPD_DEF']!=0]['SK_DPD_DEF'].sort_values(ascending=False))

1416169    3260
1047802    3229
805693     3198
1533960    3168
1060812    3137
1493959    3107
2017462    3076
55931      3045
336278     3015
3285769    2984
3424085    2954
2633713    2923
3740246    2895
1969068    2864
416409     2833
1337198    2803
798942     2800
194875     2772
2036412    2770
457321     2742
2094057    2739
2118110    2711
2445158    2708
1210886    2680
2226170    2678
2355056    2650
2421045    2650
3796470    2647
1212782    2619
2146200    2619
           ... 
2495200       1
2495204       1
2495207       1
2495417       1
2495461       1
2495477       1
2495495       1
2495541       1
2494619       1
2494535       1
2493996       1
2494485       1
2494039       1
2494041       1
2494044       1
2494047       1
2494173       1
2494193       1
2494217       1
2494238       1
2494241       1
2494250       1
2494258       1
2494262       1
2494437       1
2494443       1
2494444       1
2494448       1
2494449       1
1905402       1
Name: SK_DPD_DEF, Length

In [61]:
print(credit_card_balance[credit_card_balance['SK_DPD']!=0]['SK_DPD'].sort_values(ascending=False))

1416169    3260
1047802    3229
805693     3198
1533960    3168
1060812    3137
1493959    3107
2017462    3076
55931      3045
336278     3015
3285769    2984
3424085    2954
2633713    2923
3740246    2895
1969068    2864
416409     2833
1776114    2831
1337198    2803
1580865    2801
798942     2800
194875     2772
3443877    2770
2036412    2770
2677504    2770
339590     2770
505881     2742
457321     2742
597655     2740
2367063    2740
500987     2740
137000     2739
           ... 
1877127       1
3319940       1
3319919       1
3319915       1
1876443       1
3320125       1
1875819       1
3320212       1
1875825       1
1875846       1
1875847       1
1875860       1
1875861       1
1875862       1
1875866       1
1875873       1
1875881       1
1875934       1
1875952       1
1875966       1
1875977       1
1875989       1
1876010       1
1876014       1
1876028       1
1876042       1
1876045       1
1876232       1
1876233       1
3422044       1
Name: SK_DPD, Length: 15

In [62]:
x = set(credit_card_balance[credit_card_balance['SK_DPD_DEF']!=0].index.values)
y = set(credit_card_balance[credit_card_balance['SK_DPD']!=0].index.values)
print(x-y)

set()
{917505, 3670019, 786435, 5, 131078, 1835015, 3670024, 131079, 917512, 1966090, 3276812, 1703949, 3276814, 2097179, 1310749, 1703970, 1179683, 393253, 917542, 1703976, 917554, 2752563, 2752564, 3801141, 1835063, 1310776, 2228284, 2228290, 1966147, 1179717, 1966151, 524361, 3670092, 786509, 2228300, 2752589, 1179729, 1179730, 1179731, 3539028, 2359387, 1704027, 917596, 1704029, 3276893, 1441888, 3670115, 917613, 2359406, 1966192, 3670135, 3670136, 1966201, 1966202, 3408001, 3670145, 786563, 393345, 1966210, 917636, 3670154, 2752652, 1048719, 3670161, 1179794, 2359443, 2228370, 262293, 2097302, 2228371, 2752660, 3145883, 1179803, 1966235, 3408030, 3539103, 917669, 3408038, 2752678, 524456, 3276965, 1179820, 2228412, 3408061, 393405, 2752704, 1573057, 786626, 393412, 3145925, 131269, 3670215, 917702, 917703, 1179846, 917706, 2228427, 2752718, 3408081, 262356, 3277013, 215, 1704153, 1966297, 786655, 393443, 393444, 786662, 3277033, 2097386, 3670256, 524530, 917746, 2752755, 2752756, 

# previous

In [102]:
previous_application = pd.read_csv(PATH + 'previous_application.csv')
print(previous_application.columns.values, previous_application.shape)

['SK_ID_PREV' 'SK_ID_CURR' 'NAME_CONTRACT_TYPE' 'AMT_ANNUITY'
 'AMT_APPLICATION' 'AMT_CREDIT' 'AMT_DOWN_PAYMENT' 'AMT_GOODS_PRICE'
 'WEEKDAY_APPR_PROCESS_START' 'HOUR_APPR_PROCESS_START'
 'FLAG_LAST_APPL_PER_CONTRACT' 'NFLAG_LAST_APPL_IN_DAY' 'RATE_DOWN_PAYMENT'
 'RATE_INTEREST_PRIMARY' 'RATE_INTEREST_PRIVILEGED'
 'NAME_CASH_LOAN_PURPOSE' 'NAME_CONTRACT_STATUS' 'DAYS_DECISION'
 'NAME_PAYMENT_TYPE' 'CODE_REJECT_REASON' 'NAME_TYPE_SUITE'
 'NAME_CLIENT_TYPE' 'NAME_GOODS_CATEGORY' 'NAME_PORTFOLIO'
 'NAME_PRODUCT_TYPE' 'CHANNEL_TYPE' 'SELLERPLACE_AREA'
 'NAME_SELLER_INDUSTRY' 'CNT_PAYMENT' 'NAME_YIELD_GROUP'
 'PRODUCT_COMBINATION' 'DAYS_FIRST_DRAWING' 'DAYS_FIRST_DUE'
 'DAYS_LAST_DUE_1ST_VERSION' 'DAYS_LAST_DUE' 'DAYS_TERMINATION'
 'NFLAG_INSURED_ON_APPROVAL'] (1670214, 37)


In [111]:
print(previous_application.isnull().any())

SK_ID_PREV                     False
SK_ID_CURR                     False
NAME_CONTRACT_TYPE             False
AMT_ANNUITY                     True
AMT_APPLICATION                False
AMT_CREDIT                      True
AMT_DOWN_PAYMENT                True
AMT_GOODS_PRICE                 True
WEEKDAY_APPR_PROCESS_START     False
HOUR_APPR_PROCESS_START        False
FLAG_LAST_APPL_PER_CONTRACT    False
NFLAG_LAST_APPL_IN_DAY         False
RATE_DOWN_PAYMENT               True
RATE_INTEREST_PRIMARY           True
RATE_INTEREST_PRIVILEGED        True
NAME_CASH_LOAN_PURPOSE         False
NAME_CONTRACT_STATUS           False
DAYS_DECISION                  False
NAME_PAYMENT_TYPE              False
CODE_REJECT_REASON             False
NAME_TYPE_SUITE                 True
NAME_CLIENT_TYPE               False
NAME_GOODS_CATEGORY            False
NAME_PORTFOLIO                 False
NAME_PRODUCT_TYPE              False
CHANNEL_TYPE                   False
SELLERPLACE_AREA               False
N

In [105]:
x = previous_application.sort_values(by='SK_ID_PREV')[['SK_ID_PREV']].values
y = set(x)
print(len(x),len(y))

1670214 1670214


In [125]:
print(previous_application.groupby('NFLAG_INSURED_ON_APPROVAL')['SK_ID_PREV'].count())

NFLAG_INSURED_ON_APPROVAL
0.0    665527
1.0    331622
Name: SK_ID_PREV, dtype: int64


# bureau balance

In [127]:
bureau_balance = pd.read_csv(PATH + 'bureau_balance.csv')
print(bureau_balance.shape)
print(bureau_balance.isnull().any())

(27299925, 3)
SK_ID_BUREAU      False
MONTHS_BALANCE    False
STATUS            False
dtype: bool


In [131]:
print(bureau_balance.groupby('STATUS')['SK_ID_BUREAU'].count())

STATUS
0     7499507
1      242347
2       23419
3        8924
4        5847
5       62406
C    13646993
X     5810482
Name: SK_ID_BUREAU, dtype: int64


# bureau

In [132]:
bureau = pd.read_csv(PATH + 'bureau.csv')
print(bureau.shape)
print(bureau.isnull().any())

(1716428, 17)
SK_ID_CURR                False
SK_ID_BUREAU              False
CREDIT_ACTIVE             False
CREDIT_CURRENCY           False
DAYS_CREDIT               False
CREDIT_DAY_OVERDUE        False
DAYS_CREDIT_ENDDATE        True
DAYS_ENDDATE_FACT          True
AMT_CREDIT_MAX_OVERDUE     True
CNT_CREDIT_PROLONG        False
AMT_CREDIT_SUM             True
AMT_CREDIT_SUM_DEBT        True
AMT_CREDIT_SUM_LIMIT       True
AMT_CREDIT_SUM_OVERDUE    False
CREDIT_TYPE               False
DAYS_CREDIT_UPDATE        False
AMT_ANNUITY                True
dtype: bool


In [138]:
print(bureau[['DAYS_CREDIT','DAYS_CREDIT_ENDDATE','DAYS_ENDDATE_FACT']])

         DAYS_CREDIT  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
0               -497               -153.0             -153.0   
1               -208               1075.0                NaN   
2               -203                528.0                NaN   
3               -203                  NaN                NaN   
4               -629               1197.0                NaN   
5               -273              27460.0                NaN   
6                -43                 79.0                NaN   
7              -1896              -1684.0            -1710.0   
8              -1146               -811.0             -840.0   
9              -1146               -484.0                NaN   
10             -1146               -180.0                NaN   
11             -2456               -629.0             -825.0   
12              -277               5261.0                NaN   
13               -96                269.0                NaN   
14              -318               -187.

In [136]:
print(bureau.groupby('CREDIT_DAY_OVERDUE')['SK_ID_BUREAU'].count())

CREDIT_DAY_OVERDUE
0       1712211
1             5
2            18
3            29
4            46
5            51
6            59
7            92
8           103
9            93
10           71
11           75
12           70
13          103
14           91
15           74
16           50
17           77
18           63
19           54
20           68
21           64
22           54
23           41
24           42
25           40
26           43
27           28
28           50
29           46
         ...   
2638          1
2639          1
2640          1
2641          1
2643          1
2644          1
2649          1
2651          1
2652          1
2654          1
2656          1
2657          1
2661          1
2667          2
2677          1
2679          2
2681          1
2684          1
2685          1
2688          1
2693          1
2700          1
2703          1
2754          1
2765          1
2766          1
2770          1
2776          1
2781          1
2792          1
Name: